In [ ]:
from math import exp, log, sqrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt

from _test.linear_motion_sim import Simulator, fit_ks_and_kv, fit_ka, lognorm
from _test.func_guesser import sign

In [ ]:
tgt_v = 2

In [ ]:
def s(t):
    return 20

def a(t):
    return 22

def d(t):
    if 3 < t < 3.1:
        return 16
    
    return 0.8

sim = Simulator(max_speed_f=s, max_accel_f=a, drag_f=d, noise=0.2)

In [ ]:
class Speedometer:
    def __init__(self, num_samples):
        self.num_samples = num_samples
        self.t = []
        self.x = []


    def sample(self, t, x):
        while len(self.t) >= self.num_samples:
            del self.t[0]

        while len(self.x) >= self.num_samples:
            del self.x[0]

        self.t.append(t)
        self.x.append(x)


    def speed(self):
        if len(self.t) < 2:
            return 0
        
        return np.polyfit(self.t, self.x, 1)[0]
        

In [ ]:
sped = Speedometer(5)

def motor_power(t, x):
    sped.sample(t, x)
    v = sped.speed()
    return 0.2


t, x, v, a, p  = sim.sim(motor_power)
vt = np.ones_like(t) * tgt_v

plt.figure(figsize=(20, 6))
plt.plot(t, v)
plt.plot(t, vt, '--')
plt.show()
